Необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал> В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
URL = 'https://habr.com/ru/search/'
params = {
    'q': 'python',
    # 'q': 'data scientist'
}
req = requests.get(URL, params=params)
# req.status_code
soup = BeautifulSoup(req.text)
posts = soup.find_all('article', class_='tm-articles-list__item')
len(posts)

In [ ]:
soup

In [ ]:
posts[0]

In [ ]:
import time
def get_habr_article(query):
  habr_article = pd.DataFrame()
  URL = 'https://habr.com/ru/search/'
  for q in query:
    params = {
             'q': q
           }
    req = requests.get(URL, params=params)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    articles = soup.find_all('div', class_ = 'tm-article-snippet')
    for article in articles:
      date = article.select_one('time').get('title').split(',')[0]
      title = article.select_one('a.tm-title__link')
      if title != None:
        title = title.text
      link = article.select_one('a.tm-title__link')
      if link != None:
        link = link.get('href')
      else: 
        link = ''
      row = {'date': date, 'title': title,  'link': 'https://habr.com' + link}
      habr_article = pd.concat([habr_article, pd.DataFrame([row])])
  return habr_article.reset_index(drop=True).drop_duplicates()

In [ ]:
get_habr_article(['python'])

Функция из предыдущей части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов; в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков: <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_entire_text(url_article):
    """
    DOCSTRING: Получение полного текста статьи по ссылке со страницы поиска 
    INPUT: 'https://habr.com' + href
    OUTPUT: 
    """
    reqs = requests.get(url_article)
    soups = BeautifulSoup(reqs.text)
    return soups.find('div', class_='article-formatted-body')

In [ ]:
import time
def get_habr_search(query, page_number):
  """
  DOCSTRING: Получение дата-фрейма с новостями со страницы поиска в формате:
  <Дата - Название - Link - Likes - Полный текст>
  INPUT: список запросов, номер страницы (20 новостей на странице)
  OUTPUT: дата-фрейм
  """
  habr_article = pd.DataFrame()
  URL = 'https://habr.com/ru/search/' + 'page' + str(page_number) + '/'
  for q in query:
    params = {
            'q': q
          }
    req = requests.get(URL, params=params)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    articles = soup.find_all('article', class_='tm-articles-list__item')
    for article in articles:
      date = article.select_one('time').get('title').split(',')[0]
      title = article.select_one('a.tm-title__link')
      if title != None:
        title = title.text
      link = article.select_one('a.tm-title__link')
      if link != None:
        link = link.get('href')
      else: 
        link = ''
      likes = article.select_one('.tm-votes-meter__value').text
      contain_text = get_entire_text('https://habr.com' + link)
      if contain_text != None:
        contain_text = contain_text.text.strip().replace('\xa0', '').replace('\n', '').replace('\r', ' ')
      row = {'date': date, 'title': title,  'link': 'https://habr.com' + link, 'likes': likes, 'contain_text': contain_text}
      habr_article = pd.concat([habr_article, pd.DataFrame([row])])
  return habr_article.reset_index(drop = True)

In [ ]:
%%time
get_habr_search(['SDSJ'], 1)

In [ ]:
%%time
# df = get_multipage_search(['data scientist', 'data scientist', 'python'], 4)
# df

Обработка статей с другой разметкой - Megaposts

In [ ]:
# MEGAPOST
# https://habr.com/ru/specials/344806/


# дата   ОК

# титл 
# <h2 class="tm-megapost-snippet__title">Новый подход к спортивному анализу данных: какие шаблоны «рвет» <em class="searched-item">SDSJ</em></h2>

# href

# <a href="/ru/specials/344806/" class="tm-megapost-snippet__link tm-megapost-snippet__date"><time datetime="2017-12-25T12:07:04.000Z" title="2017-12-25, 15:07" class="tm-megapost-snippet__datetime-published">25  декабря  2017</time></a>


In [ ]:
habr_article = pd.DataFrame()
URL = 'https://habr.com/ru/search/'
params = {
         'q': 'SDSJ'
         }
req = requests.get(URL, params=params)
time.sleep(0.3)
soup = BeautifulSoup(req.text)
soup

In [ ]:
articles = soup.find_all('article', class_ = 'tm-articles-list__item')

In [ ]:
len(articles)

In [ ]:
articles[0]

In [ ]:
for article in articles:
  date = article.select_one('time').get('title').split(',')[0]
  # title = article.select_one('a.tm-title__link').text
  title = article.select_one('h2')
  if title != None:
    title = title.text
  link = article.select_one('a.tm-megapost-snippet__link')
  if link != None:
    link = link.get('href')
  else:
    link = ''  
  likes = article.select_one('.tm-votes-meter__value').text
  contain_text = get_entire_text('https://habr.com' + link)
  if contain_text != None:
    contain_text = contain_text.text.strip().replace('\xa0', '').replace('\n', '').replace('\r', ' ')
  row = {'date': date, 'title': title,  'link': 'https://habr.com' + link, 'likes': likes, 'contain_text': contain_text}
  habr_article = pd.concat([habr_article, pd.DataFrame([row])])
habr_article